In [2]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
import csv

#Sqlalchemy imports
from sqlalchemy import Column, Float, Integer, String,DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
#Clean data CSV import
hmeasures = "raw_data/clean_hawaii_measurements.csv"
hstations = "raw_data/clean_hawaii_stations.csv"
hmeasures_df = pd.read_csv(hmeasures)
hstations_df = pd.read_csv(hstations)
#Data structure overview
print(hstations_df.columns,hstations_df.dtypes) 
print("=============================")
print(hmeasures_df.columns,hmeasures_df.dtypes)

Index(['station', 'name', 'latitude', 'longitude', 'elevation'], dtype='object') station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object
Index(['station', 'date', 'prcp', 'tobs'], dtype='object') station     object
date        object
prcp       float64
tobs         int64
dtype: object


In [4]:
Base = declarative_base()

class Stations(Base):
    __tablename__ = "stations"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

class Measurements(Base):
    __tablename__ = "measurements"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(DateTime)
    prcp = Column(Float)
    tobs = Column(Float)
    

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base.metadata.create_all(engine)

#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html                   
hmeasures_df.to_sql(con=engine, index_label='id', name='measurements', if_exists='replace')
hstations_df.to_sql(con=engine, index_label='id', name='stations', if_exists='replace')

session = Session(bind=engine)


In [6]:
#Confirm table and table data import in hawaii.sqlite
inspector = inspect(engine)
inspector.get_table_names()


['measurements', 'stations']

In [10]:
data = engine.execute("SELECT * FROM stations")
for item in data:
    print(item)

(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0)
(1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6)
(2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0)
(3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9)
(4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)
(5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5)
(6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9)
(7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9)
(8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)


In [9]:
session.commit()